# MapSight_AI for Google Colab

## 概要

MapSight_AIプロジェクトをGoogle Colab環境で実行するための統合ノートブックです。
PUBGの試合動画から自動的にラウンド境界を検出し、フレーム画像を分析するための一連のツールを提供します。

### 主な機能

1. **フレーム抽出**: 動画からフレーム画像を抽出
2. **マップ切り抜き**: 透視変換によるマップ領域の抽出
3. **重複フレーム削除**: pHashアルゴリズムによる類似フレームの除去
4. **ラウンド境界検出**: テンプレートマッチング + aHashによる自動セグメンテーション

### 処理フロー

```
動画ファイル
    ↓
[フレーム抽出] → frames/
    ↓
[マップ切り抜き] → minimaps/
    ↓
[重複フレーム削除] → cleaned_frames/
    ↓
[ラウンド境界検出] → round1/, round2/, ...
```

## 1. 環境設定 (Setup Environment)

Google Driveをマウントし、プロジェクトの作業ディレクトリを設定します。

In [ ]:
# Google Driveをマウント
from google.colab import drive
import os
import shutil
from pathlib import Path

# Google Driveマウント
drive.mount('/content/drive')

# 作業ディレクトリの設定
project_root = '/content/drive/MyDrive/MapSight_AI'
data_dir = f'{project_root}/data'

# 必要なディレクトリを作成
os.makedirs(project_root, exist_ok=True)
os.makedirs(f'{data_dir}/frames', exist_ok=True)
os.makedirs(f'{data_dir}/minimaps', exist_ok=True)
os.makedirs(f'{data_dir}/templates', exist_ok=True)
os.makedirs(f'{data_dir}/deleted_frames', exist_ok=True)

# 作業ディレクトリに移動
os.chdir(project_root)

print(f"作業ディレクトリ: {os.getcwd()}")
print(f"ディレクトリ構造:")
for root, dirs, files in os.walk('.'):
    level = root.replace('.', '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files[:5]:  # 最初の5ファイルのみ表示
        print(f"{subindent}{file}")
    if len(files) > 5:
        print(f"{subindent}... and {len(files) - 5} more files")

## 2. 必要ファイルのアップロード (Upload Required Files)

以下のファイルを準備してGoogle Driveにアップロードしてください。

### 必要なファイル

1. **config.yaml** - マップ切り抜き設定ファイル
2. **テンプレート画像** - ラウンド終了画面のテンプレート
3. **サンプル動画** (オプション) - テスト用の動画ファイル

### アップロード方法

1. Google Driveの `MapSight_AI/` フォルダを開く
2. 以下のファイルをアップロード:
   - `config.yaml` → `MapSight_AI/` 直下
   - テンプレート画像 → `MapSight_AI/data/templates/`
   - 動画ファイル → `MapSight_AI/data/videos/`

In [ ]:
# デフォルトのconfig.yamlを作成
config_content = '''# MapSight_AI Configuration for Google Colab
# ROI coordinates for minimap cropping
roi:
  - [722, 0]     # 左上
  - [1800, 0]    # 右上
  - [1800, 1077] # 右下
  - [722, 1077]  # 左下

# Output size after perspective transformation
output_size: [512, 512]
'''

with open('config.yaml', 'w', encoding='utf-8') as f:
    f.write(config_content)

print("✓ config.yaml を作成しました")

# ファイル確認
if os.path.exists('config.yaml'):
    print("✓ config.yaml が存在します")
else:
    print("❌ config.yaml が見つかりません")

# テンプレートディレクトリの確認
template_dir = f'{data_dir}/templates'
if os.path.exists(template_dir):
    template_files = os.listdir(template_dir)
    if template_files:
        print(f"✓ テンプレートファイル: {template_files}")
    else:
        print("⚠ テンプレートフォルダは空です。Match_End_template.jpg をアップロードしてください")
else:
    print("❌ テンプレートディレクトリが見つかりません")

## 3. 依存関係のインストール (Install Dependencies)

MapSight_AIに必要なPythonライブラリをインストールします。

In [ ]:
# 必要なライブラリのインストール
!pip install -q opencv-python
!pip install -q numpy
!pip install -q PyYAML
!pip install -q tqdm
!pip install -q easyocr
!pip install -q Pillow
!pip install -q yt-dlp
!pip install -q ffmpeg-python

# システムレベルの依存関係
!apt-get update -qq
!apt-get install -y -qq ffmpeg

# インストール確認
import cv2
import numpy as np
import yaml
import tqdm
import time
import json
import argparse
import shutil
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed
from threading import Lock, Thread
from functools import partial
import re

try:
    import easyocr
    OCR_AVAILABLE = True
    print("✓ EasyOCR インストール済み")
except ImportError:
    OCR_AVAILABLE = False
    print("⚠ EasyOCR が利用できません")

print(f"✓ OpenCV: {cv2.__version__}")
print(f"✓ NumPy: {np.__version__}")
print("✓ 全ての依存関係がインストールされました")

## 4. Colab環境用パス修正 (Modify File Paths for Colab)

オリジナルのスクリプトをGoogle Colab環境に対応させるため、ファイルパスを修正します。

In [ ]:
# Colab環境用のパス設定
BASE_PATH = '/content/drive/MyDrive/MapSight_AI'
DATA_PATH = f'{BASE_PATH}/data'
FRAMES_PATH = f'{DATA_PATH}/frames'
MINIMAPS_PATH = f'{DATA_PATH}/minimaps'
TEMPLATES_PATH = f'{DATA_PATH}/templates'
DELETED_FRAMES_PATH = f'{DATA_PATH}/deleted_frames'

# ディレクトリ作成関数
def ensure_dirs():
    """必要なディレクトリを作成"""
    dirs = [
        FRAMES_PATH,
        MINIMAPS_PATH,
        TEMPLATES_PATH,
        DELETED_FRAMES_PATH
    ]
    for dir_path in dirs:
        os.makedirs(dir_path, exist_ok=True)
    print("✓ 必要なディレクトリを作成しました")

# 環境情報表示
def show_environment_info():
    """環境情報を表示"""
    print(f"作業ディレクトリ: {os.getcwd()}")
    print(f"Pythonバージョン: {sys.version}")
    print(f"OpenCVバージョン: {cv2.__version__}")

    # GPU情報
    try:
        import torch
        if torch.cuda.is_available():
            print(f"✓ GPU利用可能: {torch.cuda.get_device_name(0)}")
        else:
            print("⚠ GPU利用不可 (CPU処理)")
    except ImportError:
        print("⚠ PyTorch未インストール")

ensure_dirs()
import sys
show_environment_info()

## 5. フレーム抽出 (Extract Frames from Video)

動画からフレーム画像を抽出します。YouTubeのURLまたはローカル動画ファイルに対応しています。

In [ ]:
import subprocess
from datetime import datetime

def extract_frames_from_video(video_source, fps=0.5, quality=2):
    """
    動画からフレームを抽出

    Args:
        video_source (str): 動画ファイルパスまたはYouTube URL
        fps (float): フレーム抽出レート（秒あたり）
        quality (int): 画質 (2-31, 小さいほど高画質)
    """
    # 出力ディレクトリ準備
    timestamp = datetime.now().strftime('%Y-%m-%d')
    output_dir = f'{FRAMES_PATH}/{timestamp}'
    os.makedirs(output_dir, exist_ok=True)

    print(f"フレーム抽出開始...")
    print(f"入力: {video_source}")
    print(f"出力先: {output_dir}")
    print(f"FPS: {fps} (2秒に1フレーム)")

    try:
        if 'youtube.com' in video_source or 'youtu.be' in video_source:
            # YouTubeから直接抽出
            cmd = [
                'yt-dlp',
                '--no-warnings',
                '-f', 'best[ext=mp4]',
                '--exec', f'ffmpeg -i {{}} -vf "fps={fps}" -q:v {quality} "{output_dir}/frame_%05d.jpg"',
                video_source
            ]
        else:
            # ローカルファイルから抽出
            cmd = [
                'ffmpeg',
                '-i', video_source,
                '-vf', f'fps={fps}',
                '-q:v', str(quality),
                f'{output_dir}/frame_%05d.jpg'
            ]

        # コマンド実行
        result = subprocess.run(cmd, capture_output=True, text=True, cwd=BASE_PATH)

        if result.returncode == 0:
            # 生成されたフレーム数を確認
            frame_files = list(Path(output_dir).glob('*.jpg'))
            print(f"✓ フレーム抽出完了: {len(frame_files)}枚")
            print(f"出力ディレクトリ: {output_dir}")
            return output_dir
        else:
            print(f"❌ エラーが発生しました:")
            print(result.stderr)
            return None

    except Exception as e:
        print(f"❌ フレーム抽出エラー: {e}")
        return None

# フレーム抽出の実行例
video_url = ""  # ここにYouTube URLまたはファイルパスを入力

if video_url:
    extracted_frames_dir = extract_frames_from_video(video_url)
else:
    print("⚠ video_url を設定してください")
    print("例: video_url = 'https://www.youtube.com/watch?v=XXXXXXXX'")

## 6. マップ切り抜き (Crop Minimap)

透視変換を使用してフレーム画像からマップ領域を切り抜きます。

In [ ]:
def crop_minimap_from_frames(frames_dir, config_path='config.yaml'):
    """
    フレーム画像からマップ領域を切り抜く

    Args:
        frames_dir (str): フレーム画像が格納されたディレクトリ
        config_path (str): 設定ファイルのパス
    """

    # 設定ファイル読み込み
    with open(config_path, 'r', encoding='utf-8') as f:
        cfg = yaml.safe_load(f)

    # ROI座標と出力サイズ取得
    points = np.array(cfg["roi"], dtype="float32").reshape((-1, 1, 2))
    src_pts = cv2.convexHull(points).reshape(-1, 2).astype("float32")

    dst_w, dst_h = cfg["output_size"]
    dst_pts = np.array([[0, 0], [dst_w, 0], [dst_w, dst_h], [0, dst_h]], dtype="float32")

    # 透視変換行列計算
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)

    # 入出力ディレクトリ準備
    frames_path = Path(frames_dir)
    dataset_name = frames_path.name
    output_dir = Path(f'{MINIMAPS_PATH}/{dataset_name}')
    output_dir.mkdir(exist_ok=True)

    print(f"マップ切り抜き開始...")
    print(f"入力: {frames_dir}")
    print(f"出力: {output_dir}")
    print(f"ROI座標: {cfg['roi']}")
    print(f"出力サイズ: {cfg['output_size']}")

    # 画像処理ループ
    jpg_files = sorted(frames_path.glob("*.jpg"))

    if not jpg_files:
        print("❌ JPGファイルが見つかりません")
        return None

    processed_count = 0
    for f in tqdm.tqdm(jpg_files, desc="マップ切り抜き中"):
        try:
            # 画像読み込み
            img = cv2.imread(str(f))
            if img is None:
                print(f"⚠ 読み込み失敗: {f.name}")
                continue

            # 透視変換適用
            warp = cv2.warpPerspective(img, M, (dst_w, dst_h))

            # 保存
            out_f = output_dir / f.name
            cv2.imwrite(str(out_f), warp)
            processed_count += 1

        except Exception as e:
            print(f"❌ 処理エラー {f.name}: {e}")

    print(f"✓ マップ切り抜き完了: {processed_count}枚")
    print(f"出力ディレクトリ: {output_dir}")
    return str(output_dir)

# マップ切り抜きの実行
# extracted_frames_dir が設定されている場合に実行
if 'extracted_frames_dir' in locals() and extracted_frames_dir:
    cropped_maps_dir = crop_minimap_from_frames(extracted_frames_dir)
else:
    print("⚠ フレーム抽出を先に実行してください")
    # 手動でディレクトリを指定する場合
    # cropped_maps_dir = crop_minimap_from_frames('/content/drive/MyDrive/MapSight_AI/data/frames/2025-01-XX')

マップ切り抜き開始...
入力: /content/drive/MyDrive/MapSight_AI/data/frames/2025-06-10
出力: /content/drive/MyDrive/MapSight_AI/data/minimaps/2025-06-10
ROI座標: [[722, 0], [1800, 0], [1800, 1077], [722, 1077]]
出力サイズ: [512, 512]


マップ切り抜き中:   9%|▉         | 949/10160 [06:05<1:08:41,  2.23it/s]

## 7. 重複フレーム削除 (Clean Same Frames)

pHashアルゴリズムを使用して視覚的に類似したフレームを削除します。

In [ ]:
def phash64(img: np.ndarray, hash_size: int = 8) -> int:
    """64ビットpHashを計算"""
    # 1. リサイズ
    img_resized = cv2.resize(img, (hash_size * 4, hash_size * 4), interpolation=cv2.INTER_AREA)

    # 2. グレースケール変換
    if len(img_resized.shape) == 3:
        img_resized = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
    img_resized = np.float32(img_resized)

    # 3. DCT変換
    img_resized = np.ascontiguousarray(img_resized)
    dct = cv2.dct(img_resized)

    # 4. 低周波成分を取得
    dct_low = dct[:hash_size, :hash_size]
    dct_flat = np.asarray(dct_low.flatten()[1:], dtype=np.float32)  # DC成分を除外
    med = float(np.median(dct_flat))

    # 5. ハッシュ生成
    bits = (dct_flat > med).astype(np.uint8)
    bits_padded = np.pad(bits, (0, 64 - len(bits)), constant_values=0)

    # 整数に変換
    hash_int = 0
    for b in bits_padded:
        hash_int = (hash_int << 1) | int(b)
    return hash_int

def hamming_distance(a: int, b: int) -> int:
    """ハミング距離計算"""
    return (a ^ b).bit_count()

def compute_hash_colab(path: Path, hash_size: int) -> tuple:
    """ファイルのハッシュ値を計算"""
    try:
        data = np.fromfile(path, dtype=np.uint8)
        img = cv2.imdecode(data, cv2.IMREAD_COLOR)
        if img is None:
            return path, -1
        return path, phash64(img, hash_size)
    except Exception:
        return path, -1

def clean_same_frames(reference_path, target_dir, threshold=10, hash_size=8, dry_run=False):
    """
    参照画像と類似したフレームを削除

    Args:
        reference_path (str): 参照画像のパス
        target_dir (str): 対象ディレクトリ
        threshold (int): ハミング距離しきい値
        hash_size (int): ハッシュサイズ
        dry_run (bool): 実際の削除を行わない
    """

    ref_path = Path(reference_path)
    tgt_dir = Path(target_dir)

    if not ref_path.exists():
        print(f"❌ 参照画像が見つかりません: {ref_path}")
        return

    if not tgt_dir.is_dir():
        print(f"❌ 対象ディレクトリが見つかりません: {tgt_dir}")
        return

    # 参照画像のハッシュ計算
    ref_img = cv2.imread(str(ref_path))
    ref_hash = phash64(ref_img, hash_size)

    print(f"重複フレーム削除開始...")
    print(f"参照画像: {ref_path.name}")
    print(f"対象ディレクトリ: {tgt_dir}")
    print(f"しきい値: {threshold}")

    # 対象ファイル収集
    exts = {".jpg", ".jpeg", ".png"}
    files = [p for p in tgt_dir.iterdir() if p.suffix.lower() in exts]

    print(f"対象ファイル数: {len(files)}")

    if len(files) == 0:
        print("⚠ 処理対象のファイルがありません")
        return

    start_time = time.time()
    dup_paths = []

    # 並列処理でハッシュ計算
    with ProcessPoolExecutor() as pool:
        hash_args = [(f, hash_size) for f in files]
        for path, h in tqdm.tqdm(
            pool.map(lambda x: compute_hash_colab(x[0], x[1]), hash_args),
            total=len(files),
            desc="ハッシュ計算中"
        ):
            if h == -1:
                continue
            if hamming_distance(ref_hash, h) <= threshold and path != ref_path:
                dup_paths.append(path)

    # 削除ディレクトリ準備
    trash_dir = Path(DELETED_FRAMES_PATH) / tgt_dir.name
    if not dry_run:
        trash_dir.mkdir(parents=True, exist_ok=True)

    # 重複ファイル処理
    for p in dup_paths:
        if dry_run:
            print(f"[DRY-RUN] 削除対象: {p.name}")
        else:
            shutil.move(str(p), str(trash_dir / p.name))

    elapsed = time.time() - start_time
    print(f"\n✓ 処理完了")
    print(f"スキャン: {len(files)}ファイル")
    print(f"重複検出: {len(dup_paths)}ファイル")
    print(f"処理時間: {elapsed:.2f}秒")

    if not dry_run and dup_paths:
        print(f"削除先: {trash_dir}")

    return len(dup_paths)

# 使用例（適切なパスに変更してください）
# 最初のフレームを参照画像として使用
if 'cropped_maps_dir' in locals() and cropped_maps_dir:
    maps_path = Path(cropped_maps_dir)
    first_frame = next(maps_path.glob('*.jpg'), None)

    if first_frame:
        print(f"参照フレーム: {first_frame.name}")

        # ドライラン実行
        duplicate_count = clean_same_frames(
            str(first_frame),
            cropped_maps_dir,
            threshold=10,
            dry_run=True
        )

        # 実際の削除を実行する場合はコメントアウト
        # duplicate_count = clean_same_frames(
        #     str(first_frame),
        #     cropped_maps_dir,
        #     threshold=10,
        #     dry_run=False
        # )
    else:
        print("⚠ 参照フレームが見つかりません")
else:
    print("⚠ マップ切り抜きを先に実行してください")

## 8. ラウンド境界検出 (Segment Rounds)

テンプレートマッチングとaHashアルゴリズムを組み合わせてフレームをラウンド単位で分割します。

In [ ]:
def ahash_colab(path: Path, size: int = 8) -> int:
    """
    aHash (Average Hash) アルゴリズムによる画像ハッシュ計算
    """
    try:
        buf = np.frombuffer(path.read_bytes(), np.uint8)
        img = cv2.imdecode(buf, cv2.IMREAD_GRAYSCALE)
        if img is None:
            raise ValueError("imdecode failed")

        # 8x8にリサイズ
        img = cv2.resize(img, (size, size), interpolation=cv2.INTER_AREA)
        avg = img.mean()

        # 平均値以上のピクセルを1、未満を0とする二値化
        bits = 1 * (img > avg)
        return int("".join(bits.astype(int).astype(str).flatten()), 2)
    except Exception:
        return -1

def process_frame_template_colab(idx, frame_path, tmpl_gray, thr, th, tw):
    """
    テンプレートマッチング処理（単一フレーム）
    """
    try:
        buf = np.frombuffer(frame_path.read_bytes(), np.uint8)
        img = cv2.imdecode(buf, cv2.IMREAD_GRAYSCALE)
        if img is None or img.shape[0] < th or img.shape[1] < tw:
            return None

        # テンプレートマッチング
        corr = cv2.matchTemplate(img, tmpl_gray, cv2.TM_CCOEFF_NORMED)

        if corr.max() >= thr:
            return idx
        return None
    except Exception:
        return None

def segment_rounds_colab(frames_dir, template_path, segments=5, end_th=0.5, step=1, dry_run=False):
    """
    フレームをラウンド単位で分割

    Args:
        frames_dir (str): フレーム画像ディレクトリ
        template_path (str): テンプレート画像パス
        segments (int): 分割するラウンド数
        end_th (float): テンプレートマッチングしきい値
        step (int): テンプレートマッチングステップ
        dry_run (bool): 実際のファイル操作を行わない
    """

    frames_path = Path(frames_dir)
    template_path = Path(template_path)

    if not frames_path.exists():
        print(f"❌ フレームディレクトリが見つかりません: {frames_path}")
        return

    if not template_path.exists():
        print(f"❌ テンプレート画像が見つかりません: {template_path}")
        return

    # フレームファイル取得
    frames = sorted(frames_path.glob('*.jpg'))
    if not frames:
        print("❌ JPGファイルが見つかりません")
        return

    print(f"ラウンド境界検出開始...")
    print(f"フレーム数: {len(frames)}")
    print(f"テンプレート: {template_path.name}")
    print(f"分割数: {segments}")

    # テンプレート画像読み込み
    tmpl_gray = cv2.imread(str(template_path), cv2.IMREAD_GRAYSCALE)
    if tmpl_gray is None:
        print("❌ テンプレート画像の読み込みに失敗")
        return

    # aHash計算
    print("\n1. aHash計算中...")
    start_time = time.time()

    hashes = []
    for frame in tqdm.tqdm(frames, desc="aHash計算"):
        h = ahash_colab(frame)
        hashes.append(h)

    elapsed = time.time() - start_time
    print(f"✓ aHash計算完了: {elapsed:.2f}秒")

    # フレーム間ハミング距離計算
    print("\n2. フレーム間距離計算中...")
    dists = []
    for i in range(len(hashes) - 1):
        dist = hamming_distance(hashes[i], hashes[i+1])
        dists.append(dist)

    # テンプレートマッチング（簡略版）
    print("\n3. テンプレートマッチング実行中...")
    th, tw = tmpl_gray.shape[:2]
    hits = []

    # 処理対象フレームを間引き
    target_indices = range(0, len(frames), step)

    for idx in tqdm.tqdm(target_indices, desc="テンプレートマッチング"):
        result = process_frame_template_colab(idx, frames[idx], tmpl_gray, end_th, th, tw)
        if result is not None:
            hits.append(result)

    print(f"✓ テンプレートマッチング検出: {len(hits)}個")

    # 境界決定（簡略版）
    boundaries = sorted(hits)[:segments-1]  # 最大segments-1個の境界

    # 不足分をaHashで補完
    need = segments - 1 - len(boundaries)
    if need > 0:
        print(f"\n4. aHashで{need}個の境界を補完中...")
        # 距離の大きい順にソート
        candidates = sorted(range(len(dists)), key=lambda i: dists[i], reverse=True)

        for i in candidates:
            # 既存の境界から十分離れているかチェック
            if all(abs(i - b) > 50 for b in boundaries):
                boundaries.append(i)
                if len(boundaries) == segments - 1:
                    break

    boundaries = sorted(boundaries)

    # 結果表示
    segments_info = []
    prev = 0
    for b in boundaries + [len(frames)]:
        segments_info.append(b - prev)
        prev = b

    print(f"\n=== セグメンテーション結果 ====")
    print(f"総フレーム数: {len(frames)}")
    print(f"境界フレーム: {boundaries}")
    print(f"各セグメントのフレーム数: {segments_info}")

    if dry_run:
        print("\n[DRY-RUN] ファイル分割はスキップされました")
        return boundaries

    # ファイル分割実行
    print("\n5. ファイル分割実行中...")
    dataset_name = frames_path.name
    output_root = Path(f'{DATA_PATH}/{dataset_name}')
    output_root.mkdir(parents=True, exist_ok=True)

    # ラウンドディレクトリ作成
    rounds = []
    prev = 0
    for i, b in enumerate(boundaries + [len(frames)]):
        round_dir = output_root / f'round{i+1}'
        round_dir.mkdir(exist_ok=True)
        rounds.append((prev, b, round_dir))
        prev = b

    # ファイルコピー
    total_copied = 0
    for round_idx, (start_i, end_i, round_dir) in enumerate(rounds, 1):
        num_frames = end_i - start_i
        print(f"ラウンド{round_idx}処理中: {num_frames}フレーム -> {round_dir.name}/")

        for idx, frame_file in enumerate(frames[start_i:end_i], 1):
            new_name = f"Frames_{idx:05d}.jpg"
            dst = round_dir / new_name
            shutil.copy2(frame_file, dst)
            total_copied += 1

    print(f"\n✓ ファイル分割完了: {total_copied}ファイルをコピー")
    print(f"出力先: {output_root}")

    return boundaries

# ラウンド境界検出の実行
# テンプレート画像のパスを指定
template_file = f'{TEMPLATES_PATH}/Match_End_template.jpg'

if 'cropped_maps_dir' in locals() and cropped_maps_dir:
    if os.path.exists(template_file):
        print(f"使用するテンプレート: {template_file}")

        # ドライラン実行
        boundaries = segment_rounds_colab(
            cropped_maps_dir,
            template_file,
            segments=5,
            end_th=0.5,
            step=5,  # 高速化のため5フレームおき
            dry_run=True
        )

        # 実際の分割を実行する場合はコメントアウト
        # boundaries = segment_rounds_colab(
        #     cropped_maps_dir,
        #     template_file,
        #     segments=5,
        #     end_th=0.5,
        #     step=1,
        #     dry_run=False
        # )
    else:
        print(f"❌ テンプレートファイルが見つかりません: {template_file}")
        print("テンプレート画像をアップロードしてください")
else:
    print("⚠ マップ切り抜きを先に実行してください")

## 9. 環境確認と結果確認 (Environment Check & Results)

処理結果の確認と環境情報の表示を行います。

In [ ]:
def check_environment_colab():
    """Colab環境での動作確認"""
    print("=== MapSight_AI Colab環境確認 ===")

    # Python環境
    import sys
    print(f"Pythonバージョン: {sys.version}")

    # ライブラリバージョン
    print(f"OpenCV: {cv2.__version__}")
    print(f"NumPy: {np.__version__}")

    # GPU確認
    try:
        import torch
        if torch.cuda.is_available():
            print(f"GPU: {torch.cuda.get_device_name(0)}")
            print(f"CUDA: {torch.version.cuda}")
        else:
            print("GPU: 利用不可")
    except ImportError:
        print("PyTorch: 未インストール")

    # EasyOCR確認
    if OCR_AVAILABLE:
        print("EasyOCR: 利用可能")
    else:
        print("EasyOCR: 利用不可")

    # ディスク容量
    import shutil
    total, used, free = shutil.disk_usage('/content')
    print(f"ディスク容量: {free//1024//1024//1024:.1f}GB 利用可能")

check_environment_colab()

def show_results_summary():
    """処理結果のサマリー表示"""
    print("\n=== 処理結果サマリー ===")

    # データディレクトリの確認
    data_path = Path(DATA_PATH)
    if data_path.exists():
        print(f"\nデータディレクトリ: {DATA_PATH}")

        # 各サブディレクトリの確認
        subdirs = ['frames', 'minimaps', 'templates', 'deleted_frames']

        for subdir in subdirs:
            subdir_path = data_path / subdir
            if subdir_path.exists():
                # ファイル数をカウント
                jpg_files = list(subdir_path.rglob('*.jpg'))
                png_files = list(subdir_path.rglob('*.png'))
                yaml_files = list(subdir_path.rglob('*.yaml'))

                total_files = len(jpg_files) + len(png_files) + len(yaml_files)
                print(f"  {subdir}/: {total_files}ファイル")

                # サブディレクトリがある場合
                subdirs_in = [d for d in subdir_path.iterdir() if d.is_dir()]
                if subdirs_in:
                    for d in subdirs_in:
                        files_in_d = list(d.glob('*.jpg'))
                        print(f"    {d.name}/: {len(files_in_d)}ファイル")
            else:
                print(f"  {subdir}/: 見つかりません")
    else:
        print(f"データディレクトリが見つかりません: {DATA_PATH}")

    # 処理フロー状況
    print("\n=== 処理フロー状況 ===")

    # 1. フレーム抽出
    frames_dir = data_path / 'frames'
    if frames_dir.exists() and list(frames_dir.rglob('*.jpg')):
        print("✓ 1. フレーム抽出: 完了")
    else:
        print("✗ 1. フレーム抽出: 未完了")

    # 2. マップ切り抜き
    minimaps_dir = data_path / 'minimaps'
    if minimaps_dir.exists() and list(minimaps_dir.rglob('*.jpg')):
        print("✓ 2. マップ切り抜き: 完了")
    else:
        print("✗ 2. マップ切り抜き: 未完了")

    # 3. 重複フレーム削除
    deleted_dir = data_path / 'deleted_frames'
    if deleted_dir.exists() and list(deleted_dir.rglob('*.jpg')):
        print("✓ 3. 重複フレーム削除: 完了")
    else:
        print("✗ 3. 重複フレーム削除: 未完了")

    # 4. ラウンド境界検出
    round_dirs = list(data_path.glob('*/round*'))
    if round_dirs:
        print(f"✓ 4. ラウンド境界検出: 完了 ({len(round_dirs)}ラウンド)")
    else:
        print("✗ 4. ラウンド境界検出: 未完了")

show_results_summary()

# 次のステップの案内
print("\n=== 次のステップ ===")
print("1. 各セクションの変数（video_url等）を設定")
print("2. dry_run=False に変更して実際の処理を実行")
print("3. テンプレート画像の精度調整")
print("4. パラメータのチューニング（しきい値等）")
print("\n詳細な使用方法は各セクションのコメントを参照してください。")

## 使用方法と注意事項

### 基本的な使用手順

1. **セクション1-4**: 環境設定とライブラリインストール
2. **セクション5**: `video_url`変数に動画URLを設定してフレーム抽出実行
3. **セクション6**: マップ切り抜き実行
4. **セクション7**: 重複フレーム削除実行（オプション）
5. **セクション8**: テンプレート画像をアップロードしてラウンド境界検出実行

### 重要な設定項目

- `video_url`: 処理対象の動画URL（YouTube）またはファイルパス
- `template_file`: ラウンド終了画面のテンプレート画像パス
- `fps=0.5`: フレーム抽出レート（2秒に1フレーム）
- `threshold=10`: 重複フレーム検出の感度
- `end_th=0.5`: テンプレートマッチングのしきい値

### パフォーマンス最適化

- **GPU利用**: OpenCVとEasyOCRでGPU加速が自動的に有効化されます
- **並列処理**: CPUコア数に応じて自動的に並列処理が実行されます
- **メモリ管理**: 大量フレーム処理時は段階的に実行することを推奨

### トラブルシューティング

- **メモリ不足**: フレーム数が多い場合は`step`パラメータを大きくして間引き処理
- **検出精度**: テンプレートマッチングのしきい値`end_th`を調整
- **処理時間**: `dry_run=True`で事前確認してから実際の処理を実行

### ファイル構造

```
/content/drive/MyDrive/MapSight_AI/
├── config.yaml
├── data/
│   ├── frames/
│   │   └── 2025-XX-XX/
│   ├── minimaps/
│   │   └── 2025-XX-XX/
│   ├── templates/
│   │   └── Match_End_template.jpg
│   ├── deleted_frames/
│   └── 2025-XX-XX/  # 分割されたラウンド
│       ├── round1/
│       ├── round2/
│       └── ...
```

### 注意事項

- Google Colabの制限時間（12時間）内での処理完了を確認
- 大容量ファイル処理時はGoogle Driveの容量を確認
- 処理中断時は中間結果を確認して続行可能
- テンプレート画像の品質が検出精度に大きく影響